<a href="https://colab.research.google.com/github/LuccaRomagnolli/Engineering/blob/main/concrete_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Determinando a resistência a compressão do concreto por Regressão
‼️O código dever ser executado sequencialmente o clique em `Run all`

‼️Para abrir os código, por padrão ocultos, clique sobre as setas `>`

#### Downloads necesários para rodar o algorítmo

In [ ]:
# download das frameworks
! pip install kagglehub -q
! pip install pandas -q
! pip install numpy -q
! pip install matplotlib -q
! pip install scikit-learn -q

In [ ]:
# importação das frameworks
import kagglehub
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt

#### Dados

Fonte:

Proprietário Original e Doador
Prof. I-Cheng Yeh
Departamento de Gestão da Informação
Universidade Chung-Hua,
Hsin Chu, Taiwan 30067, R.O.C.
e-mail: icyeh '@' chu.edu.tw
Telefone: 886-3-5186511

Data da Doação: 3 de agosto de 2007

Origem: https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength


##### Dicionário das variáveis:


* Cement (Cimento) -- kg em uma mistura de 1 m³

* Slag (Escória de alto forno) -- kg em uma mistura de 1 m³

* Flyash (Cinzas Volantes) -- kg em uma mistura de 1 m³

* Water (Água) -- kg em uma mistura de 1 m³

* Superplasticizer (Superplastificante) -- kg em uma mistura de 1 m³

* Coarseaggregate (Agregado Graúdo) -- kg em uma mistura de 1 m³

* Fineaggregate (Agregado fino) -- kg em uma mistura de 1 m³

* Age (idade) -- Dias (1~365)

* Resistência à compressão do concreto -- MPa

In [ ]:
# Importar a base de dados
path = kagglehub.dataset_download("maajdl/yeh-concret-data")

csv_path = os.path.join(path, "Concrete_Data_Yeh.csv")
df = pd.read_csv(csv_path)


100%|██████████| 10.2k/10.2k [00:00<00:00, 3.70MB/s]

Extracting files...


In [ ]:
# Visualizar as colunas e o formato do DataFrame
print(f'Colunas: \n {list(df.columns)}')
print(f'formato do DataFrame: \n {df.shape}')

Colunas: 
 ['cement', 'slag', 'flyash', 'water', 'superplasticizer', 'coarseaggregate', 'fineaggregate', 'age', 'csMPa']
formato do DataFrame: 
 (1030, 9)


In [ ]:
# Informações técnicas do Dataframe
df.describe()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


#### Tratamento dos dados

In [ ]:
# Seperar nosso dados em Y (output:CSMPA) e X (input:características do concreto)

X = df.drop('csMPa',axis=1)
y = df['csMPa']

In [ ]:
# Preparar nossos inputs e outputs para o algorítmo de Regressão

X_train,X_test,y_train,y_test = train_test_split(X, y, random_state=42, test_size=0.2)

#### Normalização dos dados

Transformaremos os dados para que tenham média 0 e desvio padrão 1. Esse método é interessante pois diminui o custo computacional e normalmente melhora a performance do modelo.

Assim, para cada valor de `X` nas colunas, o `StandartScaler` ira achar o Z-score correspondente ao valor.

Para mais informações acesse :

 https://medium.com/@mhvasconcelos/distribui%C3%A7%C3%A3o-normal-e-a-biblioteca-standard-scaler-em-python-f21c52070c6b


In [ ]:
# Normalizar os dados (transformalos para diminuir o custo computacional e aumentar a eficiência)
# Nesse caso deixamos os dados com a média 0 e o desvio padrão 1
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# ex
X_train_scaled

array([[-1.16087734,  0.85738747,  0.9824489 , ..., -0.25746155,
        -0.64752011, -0.27567315],
       [ 1.30862304, -0.60249189,  1.23259821, ..., -1.92694961,
        -0.2731482 , -0.27567315],
       [-0.0768653 , -0.85558366,  1.06687429, ...,  1.01785948,
         0.06662828, -0.68931339],
       ...,
       [-0.86591441, -0.85558366,  1.12628475, ...,  1.34082214,
         0.33103616,  0.91561074],
       [ 1.78316909,  0.51111191, -0.83113361, ..., -1.54422615,
         0.11605031, -0.27567315],
       [ 0.28509237, -0.85558366,  0.9355459 , ..., -0.6172979 ,
         0.13458358, -0.27567315]])

#### Modelo

  Nesse projeto será usado o algoritmo  `Random Forest Regressor`. O Random Forest Regressor é um algoritmo de aprendizado supervisionado que serve para resolver problemas de regressão, ou seja, prever valores numéricos contínuos (como preços, temperaturas, etc.).

Para mais informações acesse : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [ ]:
# Importaração do modelo:

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Alimentação do modelo com dados

rf_model.fit(X_train,y_train)

RandomForestRegressor(random_state=42)

##### Medindo a correlação de cada variável em relação à compressão do concreto:

In [ ]:
print(f"\n      Importância por Variável:\n")
rf_importance_df = pd.DataFrame({
    'Variável': X.columns,
    'Correlação': rf_model.feature_importances_
}).sort_values('Correlação', ascending=False)
print(rf_importance_df.to_string(index=False))


      Importância por Variável:

        Variável  Correlação
             age    0.333812
          cement    0.325029
           water    0.126405
            slag    0.077405
superplasticizer    0.056133
   fineaggregate    0.035130
 coarseaggregate    0.028127
          flyash    0.017960


In [ ]:
# Eficácia do modelo
rf_pred_train = rf_model.predict(X_train)
rf_pred_test = rf_model.predict(X_test)

print(f' O modelo preveu com {round(r2_score(y_test,rf_pred_test),5)*100} % de acurrácia a ressistência a compressão ao concreto ')


 O modelo preveu com 88.414 % de acurrácia a ressistência a compressão ao concreto 


#### Teste Prático do Modelo

Forneça as informações solicitadas nas caixas de resposta que apareceram ao rodar o código.

`Utilize as unidades abaixo:`


1. Cement (Cimento) -- kg em uma mistura de 1 m³

2. Slag (Escória de alto forno) -- kg em uma mistura de 1 m³

3. Flyash (Cinzas Volantes) -- kg em uma mistura de 1 m³

4. Water (Água) -- kg em uma mistura de 1 m³

5. Superplasticizer (Superplastificante) -- kg em uma mistura de 1 m³

6. Coarseaggregate (Agregado Graúdo) -- kg em uma mistura de 1 m³

7. Fineaggregate (Agregado fino) -- kg em uma mistura de 1 m³

8. Age (idade) -- Dias (1~365)

9. Resistência à compressão do concreto -- MPa

In [ ]:
# Coletando os dados do usuário
input_usuário = []
for feature in X.columns:
    value = float(input(f'\n Valor para {feature}: '))
    input_usuário.append(value)

# Convertendo a lista para um formato que o modelo possa entender
user_input_df = pd.DataFrame([input_usuário], columns=X.columns)

# Fazendo a predição
prediction = rf_model.predict(user_input_df)

print(f'\nA resistência à compressão do concreto prevista é: {prediction[0]:.2f} MPa')


 Valor para cement: 200

 Valor para slag: 500

 Valor para flyash: 400

 Valor para water: 600

 Valor para superplasticizer: 900

 Valor para coarseaggregate: 800

 Valor para fineaggregate: 200

 Valor para age: 200

A resistência à compressão do concreto prevista é: 46.32 MPa
